In [60]:
from bigbio.dataloader import BigBioConfigHelpers
conhelps = BigBioConfigHelpers()
dataset = conhelps.for_config_name('cadec_bigbio_kb').load_dataset()

Reusing dataset cadec (C:\Users\richa\.cache\huggingface\datasets\cadec\cadec_bigbio_kb\1.0.0\aa0c4972a66d4e69245748c72bda714effb7d8c98aea405f186dd03cb488dbb0)


  0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'document_id': Value(dtype='string', id=None),
 'passages': [{'id': Value(dtype='string', id=None),
   'type': Value(dtype='string', id=None),
   'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
   'offsets': Sequence(feature=[Value(dtype='int32', id=None)], length=-1, id=None)}],
 'entities': [{'id': Value(dtype='string', id=None),
   'type': Value(dtype='string', id=None),
   'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
   'offsets': Sequence(feature=[Value(dtype='int32', id=None)], length=-1, id=None),
   'normalized': [{'db_name': Value(dtype='string', id=None),
     'db_id': Value(dtype='string', id=None)}]}],
 'events': [{'id': Value(dtype='string', id=None),
   'type': Value(dtype='string', id=None),
   'trigger': {'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
    'offsets': Sequence(feature=[Value(dtype='int32', id=None)], length=-1, id=None)},
 

In [64]:
len(dataset)

1

In [22]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

dataset["train"][100]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to C:\Users\richa\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline, AutoConfig
huggingface_model = 'dslim/bert-base-NER'
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(huggingface_model)
model: AutoModelForTokenClassification = AutoModelForTokenClassification.from_pretrained(huggingface_model)

In [50]:
def tokenize_function(examples):
    passages = [i[0]['text'][0] for i in examples["passages"]]
    return tokenizer(passages, padding="max_length", truncation=True)  # ["passages"][0][0]

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(600))
eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(600, 1250))

  0%|          | 0/2 [00:00<?, ?ba/s]

Loading cached shuffled indices for dataset at C:\Users\richa\.cache\huggingface\datasets\cadec\cadec_bigbio_kb\1.0.0\aa0c4972a66d4e69245748c72bda714effb7d8c98aea405f186dd03cb488dbb0\cache-65f31deff2d13078.arrow


In [52]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [56]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    print(eval_pred.shape)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [57]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [58]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: passages, coreferences, id, document_id, events, relations, entities. If passages, coreferences, id, document_id, events, relations, entities are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 600
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 225


KeyError: 'loss'